In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_columns', 200)
warnings.filterwarnings('ignore')

# Data Inspection

## 1. Overview
- `PassengerId`: the unique id of each passenger
- `Survived`: the target we need to predict:
    - 1 = Survived
    - 0 = Not Survived
- `Pclass`: the class level of each passenger:
    - 1 = Upper Class
    - 2 = Middle Class
    - 3 = Lower Class
- `Name`: passenger's name, in a format of `<Surname>, <Title>. <MiddleName LastName>`
- `Sex`: passenger's gender (male or female)
- `Age`: passenger's age
- `SibSp`:the number of each passenger's siblings and spouse
- `Parch`: the number of each passenger's parents and children
- `Ticket`: the ticket code of the passenger
- `Fare`: passenger's fare
- `Cabin`: the cabin number of the passenger, in a format of `<A-G,T>(<0-9>)+`
- `Embarked`: the port of embarkation:
    - C = Cherbourg
    - Q = Queenstown
    - S = Southampton


## 2. Missing Values


In [2]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

print("Training set missing values:")
for col in train_df.drop(columns=['Survived']).columns:
    print(f"{col}: {train_df[train_df[col].isnull()].shape[0]}")
print()
print("Test set missing values:")
for col in test_df.columns:
    print(f"{col}: {test_df[test_df[col].isnull()].shape[0]}")

## 3. Exploration

### 3.1 Pclass

From the chart shown below, we can tell people from the first class have the highest survival rate while passengers from the thrid class have the lowest survival rate.

In [3]:
sns.barplot(data=train_df,x='Pclass',y='Survived')

### 3.2 Name

We can extract titles from passengers' names, such as 'Mr.', 'Mrs', 'Miss.', 'Dr.', etc.

Here are all the unique titles I extracted from passengers' names:

{'Miss', 'the Countess', 'Jonkheer', 'Lady', 'Mme', 'Major', 'Dona', 'Ms', 'Mrs', 'Mr', 'Mlle', 'Capt', 'Rev', 'Dr', 'Master', 'Col', 'Sir', 'Don'}

In [4]:
df = pd.concat([train_df.drop(columns=['Survived']), test_df])
df['title_tmp'] = df['Name'].str.split(',').str[1]
df['Title'] = df['title_tmp'].str.strip().str.split('.').str[0]
df.drop(columns=['title_tmp'], inplace=True)
print(set(df['Title'].values))

We can have a look at the survival rate of each title:

In [5]:
train_df['Title'] = df.iloc[:train_df.shape[0]]['Title']
sns.set(rc={'figure.figsize':(23,8)})
sns.barplot(data=train_df,x='Title',y='Survived')

We can bin all the titles into 4 buckets: 'Mr', 'Mrs', 'Ms', 'Miss' and 'Special', 

where 'Special' contains: 'the Countess', 'Jonkheer', 'Lady', 'Mme', 'Major', 'Dona', 'Mlle', 'Capt', 'Rev', 'Dr', 'Master', 'Col', 'Sir', 'Don'

In [6]:
df['TitleBinned'] = df['Name'].apply(lambda x: 'Mr' if 'Mr.' in x else ('Mrs' if 'Mrs.' in x else ('Miss' if 'Miss.' in x else ('Ms' if 'Ms' in x else 'Special'))))
train_df['TitleBinned'] = df.iloc[:train_df.shape[0]]['TitleBinned']
sns.set(rc={'figure.figsize':(8,6)})
sns.barplot(data=train_df,x='TitleBinned',y='Survived')

### 3.3 SibSp and Parch

Here's the survival rate of `SibSp` and `Parch`:

In [7]:
sns.barplot(data=train_df,x='SibSp',y='Survived')

In [8]:
sns.barplot(data=train_df,x='Parch',y='Survived')

I tried to aggregate these two columns into one called `FamilySize`, which indicate the number of family members on board of each passenger.

We can see passenger with a family size of 2-4 tend to has a higher survival rate.

In [9]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
sns.barplot(data=train_df,x='FamilySize',y='Survived')

### 3.4 Ticket

There are cases that some passnegers travelled with friends or other people not from the passenger's family, which won't be told from `FamilySize`.


`Ticket` provides a way to figure this out.

We can identify passengers with the same ticket code:

In [10]:
ticket_dict = train_df['Ticket'].value_counts()
train_df['GroupSize'] = train_df['Ticket'].map(ticket_dict)
sns.barplot(data=train_df,x='GroupSize',y='Survived')

From the chart above, we can tell that passenger with a group size of 2-4 has higher survival rate.

### 3.5 Fare


The fare of each passenger has a relation to the survival possibility.

From Part 2 we can tell there are some missing values in the `Fare` column.

Since it has something to do with passenger's `Pclass`, I filled `NaN` in `Fare` column with the median number of passengers from the same `Pclass`.

Then, I binned fare values into 5 buckets:

In [11]:
from sklearn import preprocessing
# Fill NaN
pclass_set = {1, 2, 3}
median_fare_dict = train_df.groupby('Pclass')['Fare'].median().to_dict()
for pclass in pclass_set:
    train_df.loc[(train_df['Fare'].isnull()) & (train_df['Pclass'] == pclass), 'Fare'] = median_fare_dict[pclass]

In [12]:
def bin_fare(train_df, n_bins):
    # Binning
    train_df['fare_tmp'] = pd.qcut(train_df['Fare'], n_bins)
    label = preprocessing.LabelEncoder()
    train_df['FareBinned'] = label.fit_transform(train_df['fare_tmp'])
    train_df.drop(columns=['fare_tmp'], inplace=True)
    # Plotting
    sns.barplot(data=train_df,x='FareBinned',y='Survived')
    return True

In [13]:
bin_fare(train_df, 5)

We can try to bin it into various numbers of buckets.

In [14]:
bin_fare(train_df, 6)

In [15]:
bin_fare(train_df, 4)

### 3.6 Cabin


The first letter in`Cabin` number indicates the location of passenger's deck, which could be a useful factor to determine whether a passenger would survive in the end.

Here are all the `Cabin` letters in the data set: `{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', nan}`

From Part 2 we know there are lots of missing values in `Cabin`.

I will mark missing values as `U` for now, and let's see how it looks in terms of the survival rate of passengers from each deck:


In [16]:
# Some passengers might have multiple cabin numbers, I will use the last one in this case
train_df['Cabin'] = train_df['Cabin'].fillna('U')
train_df['LastCabin'] = train_df['Cabin'].str.split(' ').str[-1]
train_df['LastCabinLetter'] = train_df['LastCabin'].str[0]
train_df.loc[train_df['LastCabinLetter'] == 'T', 'LastCabinLetter'] = 'A'
sns.barplot(data=train_df,x='LastCabinLetter',y='Survived')

We can take a further look at the relation between `Cabin` and `Pclass`

In [17]:
train_df['Cabin'] = train_df['Cabin'].fillna('U')
train_df['LastCabin'] = train_df['Cabin'].str.split(' ').str[-1]
train_df['LastCabinLetter'] = train_df['LastCabin'].str[0]
for cabin in sorted(list(set(train_df['LastCabinLetter'].values))):
    print(f"Cabin {cabin}:")
    print(train_df[train_df['LastCabinLetter'] == cabin]['Pclass'].value_counts())
    print()

In conclusion:
- Deck `A`, `B`, `C`: All from class `1`
- Deck `D`: Most from class `1` and some class `2`
- Deck `E`: Most from class `1` and some from class `2`, `3`
- Deck `F`: Most from class `2` and some from class `3`
- Deck `G`: All from class `3`
- Deck `T`: All from class `1`
- Deck `U`: Most from class `3`, some from class `2`, `3`

Since there's only one passenger on deck `T`, I moved it to deck `A` instead.

### 3.7 Embarked

Here's the survival rate of passengers embarked at different port


In [18]:
sns.barplot(data=train_df[train_df['Embarked'].notnull()],x='Embarked',y='Survived')

# Feature Engineering

Based on the findings above, here's how I'm going to construct features for the training set:
- Fill `NaN` in `Cabin` columns with 'H' and covert the first letter of `Cabin` column to an integer
- Get title of each passenger, which are `Mr`, `Mrs`, `Ms`, `Miss` and `Special`, then map it to an integer
- Calculate the `FamilySize` for each passenger by: `SibSp` + `Parch` + 1
- Calculate the size of group travelling together (`GroupSize`) for each passenger since some people may travel with friends instead of with family
- Fill `NaN` in `Fare` column with the median number of passengers from the same `Pclass` and bin `Fare` column into `n` buckets
- Fill `NaN` in `Embarked` column with the most frequent value from passengers with the same `FareBinned` and `Pclass`
- Fill `NaN` in `Age` column with the median number of passengers with the same title and bin `Age` column into `n` buckets


### A final touch on the top

I've seen lots of notebooks mentioned that we could calculate a `FamilySurvival` column to the dataset.

To calculate this value, we need to get last name of each passenger, and then group the dataframe by`LastName, Fare`.

First, we set the default value of `FamilySurvival` to `0.5`.

If the number of rows in the group is greater than 1, and if there's any family member in the group survived, then set `FamilySurvival` to `1`.

If the number of rows in the group is greater than 1, and if there's no family member in the group survived, then set `FamilySurvival` to `0`.



### Pearson Correlation Heatmap

From the Pearson Correlation plot below we can tell that there are not too many features strongly correlated with one another, which means that there are not much redundant features in my training set.

The highest absolute value of correlation in from the plot:
1. `GroupSize` and `FamilySize`: 0.82
2. `Pclass` and `LastCabinLetter`: 0.75
3. `Pclass` and `FareBinned`: 0.69

In [22]:
def add_cabin_letter(df):
    """
    Covert the first letter of 'Cabin' column to an integer, and fill NaN with 0

    :param df: pd.df
    :return: pd.df with 'CabinLetter' column
    """
    df['Cabin'] = df['Cabin'].fillna('H')
    df['LastCabin'] = df['Cabin'].str.split(' ').str[-1]
    df['LastCabinLetter'] = df['LastCabin'].str[0]
    df.loc[df['LastCabinLetter'] == 'T', 'LastCabinLetter'] = 'A'
    df['LastCabinLetter'] = df['LastCabinLetter'].apply(lambda x: ord(x) - ord('A') + 1)
    df.drop(columns=['LastCabin'], inplace=True)
    return df


def add_title(df):
    """
    Get title of each passenger and map it to an integer

    :param df: pd.df
    :return: pd.df with 'Title' columns
    """
    df['TitleTmp'] = df['Name'].apply(lambda x: 'Mr' if 'Mr.' in x else ('Mrs' if 'Mrs.' in x else ('Miss' if 'Miss.' in x else 'Special')))
    label = preprocessing.LabelEncoder()
    df['Title'] = label.fit_transform(df['TitleTmp'])
    df.drop(columns=['TitleTmp'], inplace=True)
    return df


def add_family_size(df):
    """
    Calculate the size of family for each passenger

    :param df: pd.df
    :return: pd.df with 'FamilySize' column
    """
    df['FamilySize'] = df['Parch'] + df['SibSp'] + 1
    return df


def add_group_size(df):
    """
    Calculate the size of group travelling together for each passenger
    since some people may travel with friends instead of with family

    :param df: pd.df
    :return: pd.df with 'GroupSize' column
    """
    ticket_dict = df['Ticket'].value_counts()
    df['GroupSize'] = df['Ticket'].map(ticket_dict)
    return df


def process_fare(df, n_bins):
    """
    1. Fill NaN in 'Fare' column with the median number of passengers from the same 'Pclass'
    2. Bin 'Fare' column into 5 buckets

    :param df: pd.df
    :param n_bins: number of bins
    :return: pd.df with 'FareBinned' column
    """
    # Fill NaN
    pclass_set = {1, 2, 3}
    median_fare_dict = df.groupby('Pclass')['Fare'].median().to_dict()
    for pclass in pclass_set:
        df.loc[(df['Fare'].isnull()) & (df['Pclass'] == pclass), 'Fare'] = median_fare_dict[pclass]
    # Binning
    df['fare_tmp'] = pd.qcut(df['Fare'], n_bins)
    label = preprocessing.LabelEncoder()
    df['FareBinned'] = label.fit_transform(df['fare_tmp'])
    df.drop(columns=['fare_tmp'], inplace=True)
    return df


def fill_nan_in_embarked(df):
    """
    Fill NaN in 'Embarked' column with the most frequent value from passengers with the same 'FareBinned' and 'Pclass'

    :param df: pd.df
    :return: pd.df
    """
    nan_embarked_df = df[df['Embarked'].isnull()]
    for idx, row in nan_embarked_df.iterrows():
        df.loc[idx, 'Embarked'] = df[(df['Pclass'] == row['Pclass']) & (df['FareBinned'] == row['FareBinned'])]['Embarked']\
            .value_counts().idxmax()
    return df


def process_age(df, n_bins):
    """
    Fill NaN in 'Age' column with the median number of passengers with the same title
    By inspection, it is guaranteed that every title with missing 'Age' has at least one record with a valid 'Age'
    Then Bin 'Age' into 5 buckets

    :param df: pd.df
    :param n_bins: number of bins
    :return: pd.df
    """
    # Add 'title_tmp' column to get the actual title of each passenger
    df['tmp'] = df['Name'].str.split(',').str[1]
    df['title_tmp'] = df['tmp'].str.strip().str.split('.').str[0]
    # Get a df of passengers with NaN in 'Age'
    nan_age_df = df[df['Age'].isnull()]
    nan_age_titles = set(nan_age_df['title_tmp'].values)
    # Get the median number of Age from passengers with the same 'title_tmp' value
    median_age_dict = {title: df[(df['title_tmp'] == title) & (df['Age'].notnull())]['Age'].median()
                       for title in nan_age_titles}
    # Fill NaN in 'Age'
    for idx, row in nan_age_df.iterrows():
        df.loc[idx, 'Age'] = median_age_dict[row['title_tmp']]
    # Binning
    df['age_tmp'] = pd.qcut(df['Age'], n_bins)
    label = preprocessing.LabelEncoder()
    df['AgeBinned'] = label.fit_transform(df['age_tmp'])
    # Drop tmp columns
    df.drop(columns=['tmp', 'title_tmp', 'age_tmp'], inplace=True)
    return df


def fine_tuning(df):
    df['last_name'] = df['Name'].apply(lambda x:str.split(x,",")[0]) #multiple Python features in play here. How cool is that!!
    DEFAULT_SURVIVAL_VALUE = 0.5
    df['FamilySurvival'] = DEFAULT_SURVIVAL_VALUE
    for grp, grp_df in df[['Survived','Name', 'last_name', 'Fare', 'Ticket', 'PassengerId','Parch','SibSp', 'Age', 'Cabin']].groupby(['last_name','Fare']):
        if (len(grp_df) != 1): #more than one in a group   
            for ind, row in grp_df.iterrows():
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if smax == 1.0:
                    df.loc[df['PassengerId'] == passID,'FamilySurvival'] = 1
                elif (smin == 0.0):
                    df.loc[df['PassengerId']== passID, 'FamilySurvival'] = 0
    for _, grp_df in df.groupby('Ticket'):
        if (len(grp_df) != 1):
            for ind, row in grp_df.iterrows():
                if (row['FamilySurvival'] == 0) | (row['FamilySurvival'] == 0.5):
                    smax = grp_df.drop(ind)['Survived'].max()
                    smin = grp_df.drop(ind)['Survived'].min()
                    passID = row['PassengerId']
                    if (smax == 1.0):
                        df.loc[df['PassengerId'] == passID,'FamilySurvival'] = 1
                    elif(smin == 0.0):
                        df.loc[df['PassengerId'] == passID,'FamilySurvival'] = 0
    df.drop(columns=['last_name'], inplace=True)
    return df


def preprocess_df(df, fare_bins, age_bins):
    df = add_cabin_letter(df)
    df = add_title(df)
    df = add_family_size(df)
    df = add_group_size(df)
    df = process_fare(df, fare_bins)
    df = fill_nan_in_embarked(df)
    df = process_age(df, age_bins)
    df = fine_tuning(df)
    df['isMale'] = (df['Sex'] == 'male').astype(int)
    embarked_label = preprocessing.LabelEncoder()
    df['Embarked'] = embarked_label.fit_transform(df['Embarked'])
    df.drop(columns=['Name', 'PassengerId', 'Sex', 'Cabin', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Age'], inplace=True)
    return df

train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
train_n_rows = train_df.shape[0]

full_df = pd.concat([train_df, test_df])
full_df = preprocess_df(full_df, 6, 6)

train_df = full_df.iloc[:train_n_rows]
y_train = train_df['Survived'].values
X_train = train_df.drop(columns=['Survived']).values
X_test = full_df.iloc[train_n_rows:].drop(columns=['Survived']).values

colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_df.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

# Model Selection

For this kind of binary classification task, KNN, random forest, XGBoost and logistic regression algorithms are usually used.

I've used a grid search/random search to tune hyper-parameters for each model, and select the one with the highest ROC_AUC score.

### 1. KNN

In [23]:
hyperparams = {
    'algorithm': ['auto'], 
    'weights': ['uniform','distance'], 
    'leaf_size': [i for i in range(1, 55, 5)],
    'n_neighbors': [i for i in range(4, 50, 2)]
}
knn_grid_search = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=hyperparams,
    verbose=True,
    cv=10,
    n_jobs=-1,
    scoring="roc_auc"
)
knn_grid_search.fit(X_train,y_train)

print("KNN")
print("Best parameters: ", knn_grid_search.best_estimator_)
print("Best ROC_AUC: ", knn_grid_search.best_score_)

### 2. Radom Forest

In [ ]:
hyperparams = {
    'n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],
    'max_features': ['auto', 'sqrt', 'log2'] + [i for i in range(1, 26)],
    'max_depth': [i for i in range(2, 500, 2)] + [None],
    'min_samples_split': [i for i in range(2, 200, 2)],
    'max_leaf_nodes': [i for i in range(2, 2000, 4)],
    'max_samples': [i * 0.1 for i in range(1, 10)],
    'bootstrap': [True]
}
rf = RandomForestClassifier()
rf_randsearch = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=hyperparams, 
    n_iter=100, 
    cv=10,
    random_state=42, 
    n_jobs=-1,
    scoring='roc_auc'
)
rf_randsearch.fit(X_train, y_train)
print("Random Forest")
print("Best parameters: ", rf_randsearch.best_estimator_)
print("Best ROC_AUC: ", rf_randsearch.best_score_)

### 3. XGBoost

In [ ]:
hyperparams = {
    'max_depth': [3, 6, 10, 20, 40, 80],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8],
    'n_estimators': [100, 500, 1000, 2000],
    'colsample_bytree': [0.1, 0.3, 0.7, 1]
}
xgb_model = xgb.XGBClassifier(objective="binary:logistic", verbosity=0)
xgb_randsearch = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=hyperparams,
    scoring='roc_auc',
    cv=10,
    n_iter=100,
    n_jobs=-1
)
xgb_randsearch.fit(X_train, y_train)
print("XGBoost")
print("Best parameters: ", xgb_randsearch.best_estimator_)
print("Best ROC_AUC: ", xgb_randsearch.best_score_)

### 4. Logistic Regression

#### 4.1 L2 Regularization (Ridge Regression)

In [ ]:
l2_grid = {
    'penalty': ['l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [i for i in range(100, 5000, 100)],
    'warm_start': [True, False],
    'class_weight': [None, 'auto'],
    'C': [i * 0.1 for i in range(1, 50)]
}
l2 = LogisticRegression()
l2_randsearch = RandomizedSearchCV(
    estimator=l2, 
    param_distributions=l2_grid, 
    n_iter=100, 
    cv=3,
    random_state=42,
    n_jobs=-1,
    scoring='roc_auc'
)
l2_randsearch.fit(X_train, y_train)
print("Logistic Regression with L2 Regularization")
print("Best parameters: ", l2_randsearch.best_estimator_)
print("Best ROC_AUC: ", l2_randsearch.best_score_)

#### 4.2 L1 Regularization (Lasso Regression)

In [ ]:
l1_grid = {
    'penalty': ['l1'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [i for i in range(100, 5000, 100)],
    'warm_start': [True, False],
    'class_weight': [None, 'auto'],
    'C': [i * 0.1 for i in range(1, 50)]
}
l1 = LogisticRegression()
l1_randsearch = RandomizedSearchCV(
    estimator=l1, 
    param_distributions=l1_grid, 
    n_iter=100, 
    cv=3,
    random_state=42, 
    n_jobs=-1,
    scoring='roc_auc'
)
l1_randsearch.fit(X_train, y_train)
print("Logistic Regression with L1 Regularization")
print("Best parameters: ", l1_randsearch.best_estimator_)
print("Best ROC_AUC: ", l1_randsearch.best_score_)

#### 4.3 L1 & L2 Regularization

In [ ]:
l12_grid = {
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'max_iter': [i for i in range(100, 5000, 100)],
    'warm_start': [True, False],
    'class_weight': [None, 'auto'],
    'C': [i * 0.1 for i in range(1, 50)],
    'l1_ratio': [i * 0.1 for i in range(1, 10)]
}
l12 = LogisticRegression()
l12_randsearch = RandomizedSearchCV(
    estimator=l12, 
    param_distributions=l12_grid, 
    n_iter=100, 
    cv=10,
    random_state=42, 
    n_jobs=-1,
    scoring='roc_auc'
)
l12_randsearch.fit(X_train, y_train)
print("Logistic Regression with L1 & L2 Regularization")
print("Best parameters: ", l12_randsearch.best_estimator_)
print("Best ROC_AUC: ", l12_randsearch.best_score_)

# Generate Submission File

In [ ]:
def generate_submission_file(model, X_train, y_train, X_test, filename):
    print(f"Generating {filename} ...")
    model.best_estimator_.fit(X_train, y_train)
    y_pred = model.best_estimator_.predict(X_test)
    output_df = pd.DataFrame(pd.read_csv('titanic/test.csv')['PassengerId'])
    output_df['Survived'] = y_pred.astype(int)
    output_df.to_csv(filename, index=False)
    print("Done")
    return True

model_fn_lst = [
    [knn_grid_search, 'submission/KNN_submission.csv'],
    [xgb_randsearch, 'submission/XGBoost_submission.csv'],
    [rf_randsearch, 'submission/RandomForest_submission.csv'],
    [l2_randsearch, 'submission/RidgeRegression_submission.csv'],
    [l1_randsearch, 'submission/LassoRegression_submission.csv'],
    [l12_randsearch, 'submission/L12Regression_submission.csv']
]
for model, out_fn in model_fn_lst:
    generate_submission_file(model, X_train, y_train, X_test, out_fn)